In [102]:
# import visualization packages
from polygon import RESTClient
import plotly.graph_objects as go
from plotly.offline import plot
import plotly.express as px

# import pandas and datetime
import pandas_market_calendars as mcal
import pandas as pd
import datetime as dt

# import API_KEY from config
from config import *

In [103]:
client = RESTClient(API_KEY)

In [240]:
ticker = 'AAPL'
earnings_date = '2022-10-27'
expiration_date = '2022-11-04'

In [105]:
# get the date that is 30 days before expiration
start_date = (pd.to_datetime(earnings_date) - dt.timedelta(days=30)).date().strftime('%Y-%m-%d')

In [106]:
# create a dataframe of contracts available as of the date specified in 'contracts_as_of'
contract_names = []
for c in client.list_options_contracts(ticker, limit=1000, as_of=start_date):
    contract_names.append(c)

df = []
for c in contract_names:
    df.append({
        'expiration date': c.expiration_date,
        'type': c.contract_type,
        'strike price': c.strike_price,
        'option ticker': c.ticker
    })
    
df = pd.DataFrame(df)
df['expiration date'] = pd.to_datetime(df['expiration date'])

In [118]:
# get the price of the underlying
underlying = client.get_aggs(ticker=ticker, from_=start_date, to=earnings_date, multiplier=1, timespan='day')
underlying = pd.DataFrame(underlying)
underlying['date'] = pd.to_datetime(underlying['timestamp'] * 1000000)

In [258]:
expiration_mask = df['expiration date'] == expiration_date
call_mask = df['type'] == 'call'
put_mask = df['type'] == 'put'
min_value_mask = round(abs(df[expiration_mask & call_mask]['strike price'] - underlying['close'].iloc[0]).min(), 2)

In [255]:
stock_price = underlying['close'].iloc[0]
df.iloc[abs(df[expiration_mask]['strike price'] - stock_price).sort_values().head(2).index]

,expiration date,type,strike price,option ticker
649,2022-11-04,call,152.5,O:AAPL221104C00152500
702,2022-11-04,put,152.5,O:AAPL221104P00152500


In [271]:
call = df.iloc[abs(df[expiration_mask & call_mask]['strike price'] - stock_price).sort_values().head(1).index]
put = df.iloc[abs(df[expiration_mask & put_mask]['strike price'] - stock_price).sort_values().head(1).index]

In [488]:
def get_net_pnl(call_df, put_df, start_date):
    call = client.get_aggs(
        ticker = call_df['option ticker'].iloc[0],
        limit=50000,
        multiplier = 1, 
        timespan = 'day', 
        from_ = start_date, 
        to = earnings_date)
    put = client.get_aggs(
        ticker = put_df['option ticker'].iloc[0],
        limit=50000,
        multiplier = 1, 
        timespan = 'day', 
        from_ = start_date, 
        to = earnings_date)
    call = pd.DataFrame(call)
    call['date'] = pd.to_datetime(call['timestamp'] * 1000000)

    put = pd.DataFrame(put)
    put['date'] = pd.to_datetime(put['timestamp'] * 1000000)
    
    net_df = pd.DataFrame()
    net_df['close'] = call['close'] + put['close']
    col_name = f'{len(net_df)} days'
    net_df['date'] = call['date']
    net_df[col_name] = round(((net_df['close'] - net_df['close'].iloc[0]) / net_df['close'].iloc[0]) * 100, 2)
    net_df.drop('close', axis=1, inplace=True)
    return net_df

In [491]:
net_df = get_net_pnl(call, put, start_date)

In [492]:
net_df2 = get_net_pnl(call, put, '2022-09-30')

In [503]:
all_df = pd.merge(net_df, net_df2, how='outer', on='date').fillna(0)

In [504]:
all_df.set_index('date', inplace=True)

In [505]:
list_of_pnl = []

In [506]:
for col in all_df.columns:
    list_of_pnl.append(all_df[col].to_list())

In [507]:
fig = px.imshow(all_df.transpose(), color_continuous_scale=[(0,'red'), (0.5,'white'), (1.0, 'green')], range_color=(-100 ,100))
fig.show()

# Find valid trading dates

In [296]:
nyse = mcal.get_calendar('NYSE')

In [99]:
valid_dates = nyse.valid_days(start_date=start_date, end_date=earnings_date)
valid_dates = [date.date().strftime('%Y-%m-%d') for date in valid_dates]

In [289]:
valid_dates

['2022-09-27',
 '2022-09-28',
 '2022-09-29',
 '2022-09-30',
 '2022-10-03',
 '2022-10-04',
 '2022-10-05',
 '2022-10-06',
 '2022-10-07',
 '2022-10-10',
 '2022-10-11',
 '2022-10-12',
 '2022-10-13',
 '2022-10-14',
 '2022-10-17',
 '2022-10-18',
 '2022-10-19',
 '2022-10-20',
 '2022-10-21',
 '2022-10-24',
 '2022-10-25',
 '2022-10-26',
 '2022-10-27']